In [43]:
import pandas as pd
from tqdm import tqdm
tax_info = pd.read_csv('../data/train/tax_info.csv')
entprise_info = pd.read_csv('../data/train/entprise_info.csv')

In [44]:
print(set(tax_info.TAX_CATEGORIES))
print(tax_info.isna().sum())
tax_info.nunique()



{'城镇土地使用税', '耕地占用税', '个人所得税', '其他专项收入', '其他收入', '地方教育附加', '教育费附加', '房产税', '企业所得税', '城市维护建设税', '残疾人就业保障金', '土地增值税', '营业税', '印花税', '水利建设专项收入', '税务部门罚没收入', '契税'}
id                   0
START_DATE           0
END_DATE             0
TAX_CATEGORIES       0
TAX_ITEMS            0
TAXATION_BASIS    3379
TAX_RATE          3379
DEDUCTION         4960
TAX_AMOUNT           0
dtype: int64


id                 808
START_DATE          91
END_DATE            91
TAX_CATEGORIES      17
TAX_ITEMS          275
TAXATION_BASIS    1665
TAX_RATE            28
DEDUCTION          247
TAX_AMOUNT        4568
dtype: int64

In [45]:
#处理tax数据
#对object类型进行编码
tax_info_clean=tax_info.drop(['START_DATE','END_DATE'],axis=1)
tax_info_clean['TAX_CATEGORIES']=tax_info_clean['TAX_CATEGORIES'].fillna(-1)
tax_info_clean['TAX_ITEMS']=tax_info_clean['TAX_ITEMS'].fillna(-1)
cat_cols = ['TAX_CATEGORIES','TAX_ITEMS']

for cat_col in tqdm(cat_cols):
    tax_info_clean[cat_col] = tax_info_clean[cat_col].astype('category').cat.codes

num_cols = ['TAX_AMOUNT','TAX_RATE','DEDUCTION']
tax_info_clean=tax_info_clean.fillna(-1)
#
count_grouped = tax_info_clean.groupby('id',sort=False)[cat_cols].count()
cat_count_feature = count_grouped.reindex()

num_grouped = tax_info_clean.groupby('id',sort=False)[num_cols].agg('mean')
num_count_feature = num_grouped.reindex()

tax_info_clean = pd.merge(num_count_feature,cat_count_feature, on = 'id',how = 'outer')
tax_info_clean.info()

100%|██████████| 2/2 [00:00<00:00, 61.68it/s]<class 'pandas.core.frame.DataFrame'>
Index: 808 entries, f000950527a6feb6c2f40c9d8477e73a439dfa0897830397 to d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   TAX_AMOUNT      808 non-null    float64
 1   TAX_RATE        808 non-null    float64
 2   DEDUCTION       808 non-null    float64
 3   TAX_CATEGORIES  808 non-null    int64  
 4   TAX_ITEMS       808 non-null    int64  
dtypes: float64(3), int64(2)
memory usage: 37.9+ KB



In [46]:
tax_info_clean.to_csv('tax_info_feature.csv')